In [114]:
import numpy as np
import pandas as pd
from collections import defaultdict

from train_utils import datasets, train_and_eval_model, run_hparam_search, select_features_with_lasso

from xgboost import XGBRegressor, XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler


In [121]:
admissions = pd.read_csv("mimic_admissions.csv").drop("hadm_id", axis=1)
diagnoses = pd.read_csv("mimic_diagnoses_icd.csv").drop("hadm_id", axis=1)
discharge = pd.read_csv("mimic_discharge_summaries.csv").drop("hadm_id", axis=1)
ed = pd.read_csv("mimic_ed_visits.csv").drop(["edregtime", "edouttime", "hadm_id"], axis=1)
lace = pd.read_csv("mimic_lace_scores.csv").drop("hadm_id", axis=1)

merged_df = pd.concat(objs=[admissions, diagnoses, discharge, ed, lace], axis=1, join="outer")

merged_df.to_csv("merged_2.csv")


In [125]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Load the dataset
# df = pd.read_csv("merged_data.csv")
df = pd.read_csv("merged_2.csv")


# Print column names to debug
print("Columns in dataset:", df.columns)

# Check unique values in readmission_risk
print("Unique values in readmission_risk:", df["readmission_risk"].unique())

# Map categorical readmission risk to numerical values
risk_mapping = {"Moderate": 0, "High": 1}  # Adjust mapping if needed
df["readmission_risk"] = df["readmission_risk"].map(risk_mapping)

# Drop any NaN values resulting from mapping
df = df.dropna(subset=["readmission_risk"])

# Ensure the dataset isn't empty after processing
if df.empty:
    raise ValueError("Dataset is empty after processing. Check readmission_risk values.")

# Identify categorical features
cat_features = df.select_dtypes(include=["object"]).columns.tolist()

# Encode categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features)
    ],
    remainder='passthrough'
)

# Select features and target variable
# X = df.drop(columns=["readmission_risk", "subject_id", "admittime", "dischtime", "deathtime", "edregtime", "edouttime", "text", "hadm_id"])
X = df.drop("readmission_risk", axis=1)
y = df["readmission_risk"]

# Convert target to numeric if needed
y = pd.to_numeric(y, errors='coerce')


# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)


# Ensure data is not empty after split
if len(X_train) == 0 or len(X_test) == 0:
    raise ValueError("Training or testing set is empty. Adjust test_size or check data filtering.")

# Train XGBoost model
model = xgb.XGBClassifier(objective="multi:softmax", num_class=3, eval_metric="mlogloss", random_state=42)
model.fit(X_train, y_train)


# Make predictions
y_pred = model.predict(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")


/var/folders/9v/_3psvbl52k53mrrcnt2q29rr0000gq/T/ipykernel_65133/2663262777.py:9: DtypeWarning: Columns (2,3,5,6,7,14,16,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("merged_2.csv")


Columns in dataset: Index(['Unnamed: 0', 'subject_id', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'edregtime', 'edouttime', 'los_days', 'subject_id.1',
       'icd_code', 'icd_version', 'diagnosis_description', 'subject_id.2',
       'text', 'subject_id.3', 'disposition', 'subject_id.4', 'l_score',
       'a_score', 'c_score', 'e_score', 'lace_total', 'readmission_risk'],
      dtype='object')
Unique values in readmission_risk: ['Moderate' 'High' nan]
Model Accuracy: 1.0000
